In [ ]:
!pip install -q git+https://github.com/gjoseph92/stackstac.git@main
!pip install -q pystac==1.2

In [ ]:
import pystac
import stackstac
import geopandas as gpd
import hvplot.xarray
from dask.distributed import Client
import os

In [ ]:
# GDAL environment variables for better performance

os.environ['AWS_REGION']='us-west-2'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' 
os.environ['AWS_NO_SIGN_REQUEST']='YES' 

In [ ]:
# Paste /proxy/localhost:8787 for cluster diagnostics
client = Client(local_directory='/tmp', processes=False)
client

In [ ]:
# Load STAC ItemCollection
stac_items = pystac.ItemCollection.from_file('mycollection.json')
items = [item.to_dict(transform_hrefs=False) for item in stac_items]

stack = stackstac.stack(items,
                        dtype='float32')
stack

In [ ]:
gamma0_vv = stack.sel(band='gamma0_vv')

gamma0_vv.hvplot.image(x='x', y='y',
                      rasterize=True,
                      data_aspect=1,
                      title='gamma0_vv',
                      clim=(0,0.4), # not equal to vv scale
                      cmap='gray',
                      clabel='gamma0_vv (watts)')

In [ ]:
snotel = {'622_CO_SNTL': dict(longitude=-108.058350, latitude=39.058311, elevation=3048.0),
          '682_CO_SNTL': dict(longitude=-107.874138, latitude=39.046440, elevation=3035.808105)
         }
df = gpd.pd.DataFrame(snotel).T

geometries = gpd.points_from_xy(df.longitude.values, df.latitude.values)

gf = gpd.GeoDataFrame(df, geometry=geometries, crs="EPSG:4326")

gf

In [ ]:
gf_utm = gf.to_crs(stack.crs)
gf_utm

In [ ]:
easting, northing = gf_utm.loc['622_CO_SNTL'].geometry.coords[0]
#easting # 754536.6884321134
#northing # 4327366.69808165

In [ ]:
# Get timeseries at a point
ts = stack.sel(band='gamma0_vv').sel(x=easting,y=northing, method='nearest').compute()

In [ ]:
ts.hvplot.scatter(x='time', ylabel='gamma0_vv (watts)')

In [ ]:
#selection based on coordinate values:
ts.hvplot.scatter(x='time', c='platform', ylabel='gamma0_vv (watts)')

In [ ]:
ts.hvplot.scatter(x='time', c='sat:orbit_state', ylabel='gamma0_vv (watts)')

In [ ]:
# work with small box around flat area of mesa
buffer = 100 # meters
x = 7.403e5
y = 4.327e6
ts_box = stack.sel(band='gamma0_vh').sel(x=slice(x-buffer,x+buffer), y=slice(y+buffer, y-buffer))

# can work around this by pulling all data into memory first
# NotImplementedError: The da.nanmedian function only works along an axis or a subset of axes.  The full algorithm is difficult to do in parallel
#ts = ts_box.median().compute()
ts = ts_box.compute().median(dim=['x','y'])

In [ ]:
ts_box

In [ ]:
line = ts.where(~ts.isnull(), drop=True).hvplot.line(x='time', c='k')
points = ts.hvplot.scatter(x='time', c='sat:orbit_state', ylabel='gamma0_vh (watts)')
line * points

In [ ]:
%%time 
# Compare VV and VH in same plot

box = stack.sel(band=['gamma0_vv','gamma0_vh']).sel(x=slice(x-buffer,x+buffer), y=slice(y+buffer, y-buffer)).compute()

In [ ]:
%%time

ts_vv = box.sel(band='gamma0_vv').median(dim=['x','y'])
ts_vh = box.sel(band='gamma0_vh').median(dim=['x','y'])

line_vv = ts_vv.where(~ts_vv.isnull(), drop=True).hvplot.line(x='time', c='k')
line_vh = ts_vh.where(~ts_vh.isnull(), drop=True).hvplot.line(x='time', c='m')

points_vv = ts_vv.hvplot.scatter(x='time', c='k', label='VV')
points_vh = ts_vh.hvplot.scatter(x='time', c='m', label='VH')

(line_vv * line_vh * points_vv * points_vh).opts(ylabel='backscatter power (watts)')

In [ ]:
ts_vv_norm = ts_vv/ts_vv.max()
ts_vh_norm = ts_vh/ts_vh.max()

line_vv = ts_vv_norm.where(~ts_vv.isnull(), drop=True).hvplot.line(x='time', c='k', label='VV')
line_vh = ts_vh_norm.where(~ts_vh.isnull(), drop=True).hvplot.line(x='time', c='m', label='VH')

(line_vv * line_vh).opts(ylabel='normalized backscatter power (watts)')